# Simulate 1-year Projections

Using all that we have determined in the scripts leading up to this one, we will use the built-in simulation method in SARIMAX library to simulate projections. This will allow us to determine prediction quantiles, in addition to the 95% CI we report on.

## Step 1: Import data

* total felony crimes

* total felony arrests

* admission counts

* 30-day average jail population

In [502]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.api import SARIMAX

In [503]:
monthly_pop = pd.read_csv("../Data/_30_day_adp.csv", index_col=0)
joined_doc_ivs_df = pd.read_csv("../Data/_30_day_IVs.csv", index_col = 0)
crime_data = pd.read_csv("../Data/_30_day_crime_counts.csv", index_col = 0)
arrest_data = pd.read_csv("../Data/_30_day_arrest_counts.csv", index_col = 0)

In [504]:
monthly_pop['ln_adp'] = np.log(monthly_pop['ADP'])
monthly_pop.head()

,Start Date,End Date,ADP,ln_adp
0,2016-05-30,2016-06-28,9813.0,9.191463
1,2016-06-29,2016-07-28,9748.0,9.184817
2,2016-07-29,2016-08-27,9767.0,9.186765
3,2016-08-28,2016-09-26,9882.0,9.198470
4,2016-09-27,2016-10-26,9810.0,9.191158


In [505]:
adm_df = joined_doc_ivs_df[['Start Date', 'End Date', 'admission_count']]
pd_data = crime_data[['Start Date', 'End Date', 'total_felony_crimes']].merge(arrest_data[['Start Date', 'End Date', 'total_felony_arrest']], left_on = ['Start Date', 'End Date'], right_on = ['Start Date', 'End Date'])
adm_df['ln_adm'] = np.log(adm_df['admission_count'])
pd_data['ln_crime'] = np.log(pd_data['total_felony_crimes'])
pd_data['ln_arrest'] = np.log(pd_data['total_felony_arrest'])
adm_df.head()

C:\Users\emjoh\AppData\Local\Temp\ipykernel_1416\2470668559.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  adm_df['ln_adm'] = np.log(adm_df['admission_count'])


,Start Date,End Date,admission_count,ln_adm
0,2016-05-30,2016-06-28,4825,8.481566
1,2016-06-29,2016-07-28,4774,8.470940
2,2016-07-29,2016-08-27,4783,8.472823
3,2016-08-28,2016-09-26,4672,8.449343
4,2016-09-27,2016-10-26,4619,8.437934


In [506]:
adm_df.tail(1)

,Start Date,End Date,admission_count,ln_adm
96,2024-04-18,2024-05-17,1982,7.591862


In [507]:
pd_data.head()

,Start Date,End Date,total_felony_crimes,total_felony_arrest,ln_crime,ln_arrest
0,2016-05-30,2016-06-28,12730,7630,9.451717,8.939843
1,2016-06-29,2016-07-28,13042,7866,9.475930,8.970305
2,2016-07-29,2016-08-27,13173,7918,9.485925,8.976894
3,2016-08-28,2016-09-26,12678,7505,9.447623,8.923325
4,2016-09-27,2016-10-26,12661,8091,9.446282,8.998508


In [508]:
pd_data.tail(1)

,Start Date,End Date,total_felony_crimes,total_felony_arrest,ln_crime,ln_arrest
94,2024-02-18,2024-03-18,13481,8837,9.509037,9.086703


## Step 2: Fitting and Predicting Exogenous Variables

In [509]:
#define the model parameters based on our analysis
arima_dict = {
    "admission_count": {"order": [0, 1, 0], "seasonal_order": [1, 0, 0, 12]},
    "total_felony_crimes": {"order": [0, 1, 0], "seasonal_order": [2, 0, 0, 12]},
    "total_felony_arrest": {"order": [2, 1, 0], "seasonal_order": [1, 0, 0, 12]},
    "ADP": {"order": [2, 2, 0], "seasonal_order": [0, 0, 0, 12]}
}

In [510]:
#determine prediction size for each consecutive iteration based on 12 mo future predicts plus any missing data
missing_n_crime = 0 if len(monthly_pop) < len(pd_data) else len(monthly_pop) - len(pd_data)
missing_n_arrest = 0 if len(monthly_pop) < len(pd_data) else len(monthly_pop) - len(pd_data)
missing_n_adm = 0 if len(monthly_pop) < len(adm_df) else len(monthly_pop) - len(adm_df)


In [511]:
#step 2a: felony crimes
# order = arima_dict['total_felony_crimes']['order']
# seasonal_order = arima_dict['total_felony_crimes']['seasonal_order']
order = (0, 1, 0)
seasonal_order = (2, 0, 0, 12)

y = pd_data['ln_crime'].dropna()
model = SARIMAX(y, order=order, seasonal_order=seasonal_order)
model_fit = model.fit(disp=False)
#determine predictions lengths and save the output in dictionary
n_steps = 12 + missing_n_crime
# Generate in-sample predictions
y_pred = model_fit.fittedvalues
# Generate out-of-sample forecast_exog
forecast = model_fit.get_forecast(steps=n_steps)
y_forecast_log = forecast.predicted_mean
y_forecast = np.exp(y_forecast_log)

#store forecast in the auto arima dictionary
arima_dict['total_felony_crimes']['y'] = np.exp(y)
arima_dict['total_felony_crimes']['prediction'] = y_forecast
print(model_fit.summary())

                                     SARIMAX Results                                      
Dep. Variable:                           ln_crime   No. Observations:                   95
Model:             SARIMAX(0, 1, 0)x(2, 0, 0, 12)   Log Likelihood                 138.316
Date:                            Fri, 21 Jun 2024   AIC                           -270.632
Time:                                    15:36:15   BIC                           -263.003
Sample:                                         0   HQIC                          -267.551
                                             - 95                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.S.L12       0.1905      0.080      2.393      0.017       0.034       0.346
ar.S.L24       0.1660      0.121   

c:\Users\emjoh\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [512]:
#simulate the predictions and we will use them in simulating the predictions for total felony arrests
crime_simulation = np.exp(model_fit.simulate(nsimulations=2500, repetitions= n_steps, anchor = 'end'))
crime_simulation.describe()

ln_crime                                                           
                 0             1             2             3             4    
count   2500.000000   2500.000000   2500.000000   2500.000000   2500.000000  \
mean     922.096480    530.602702  21791.715913  13198.175183  13216.494104   
std     1735.473461   1408.821022  22119.308597  18584.891091  15435.635951   
min       36.597307      2.377836   1426.333340     44.434699    394.413856   
25%      225.134058     40.737279   4989.206204    403.325117   3208.160797   
50%      471.924806    184.744395  13054.405720   2895.585114   7758.739456   
75%      848.645196    355.513571  29938.178881  20554.645743  18237.331499   
max    15653.627673  14266.661326  93591.199500  93294.860360  95470.431890   

                                                                               
                 5              6             7             8             9    
count  2.500000e+03    2500.000000  2.500000e+03   2500.000000   2500.000000  \
mean   4.335043e+05   21230.655042  4.597310e+06   5682.371016   2136.145348   
std    5.103990e+05   26988.411742  9.035130e+06   4584.354379   2573.887750   
min    8.043145e+03    1133.362936  2.898648e+03    120.068651    111.103463   
25%    7.624790e+04    4831.321699  2.223325e+04   2245.963026    581.031189   
50%    1.796496e+05    9528.455360  6.517228e+04   4507.980903   1455.385297   
75%    7.598190e+05   26664.371580  5.458249e+06   7990.803753   2734.129218   
max    2.069015e+06  151686.690738  5.379179e+07  21651.449900  18710.711738   

                                                                              
                 10            11             12            13            14  
count  2.500000e+03   2500.000000    2500.000000   2500.000000   2500.000000  
mean   2.444955e+05   5735.672656   13268.018957  11385.102519  15997.701653  
std    4.226063e+05   4675.068642   17460.227301   8427.785707  17087.429028  
min    2.142201e+03    275.934228     321.866886    320.214836    700.951108  
25%    1.160385e+04   1845.675883    2463.978728   5062.716264   4132.285859  
50%    4.058339e+04   4507.553792    6708.995958   9131.212917  11436.996173  
75%    2.423817e+05   8503.214124   15363.908680  17158.055306  21056.719376  
max    2.177164e+06  21502.159661  112982.265990  42091.866250  99322.738176

In [513]:
crime_simulation.sample(5)

ln_crime                                                         
               0           1             2             3             4    
849    928.855545  164.963982   8151.877823  11724.080445   8607.789922  \
1149   861.707755  111.941092  30789.397466   4195.201294  43324.934526   
368    602.440787  728.821439   2942.196738  31651.807820  26162.077264   
2350  1017.766378   15.849629   4989.531848     82.813361   5095.674445   
564    299.987257  856.293683   3143.479638  29150.422140  13100.081682   

                                                                            
                 5             6             7            8            9    
849   252708.335372  20622.100882  3.503534e+04  1326.383790  3372.169184  \
1149   55961.282006  41530.744399  6.932865e+04  2744.385617   147.125210   
368    25310.760770  23830.076564  6.449830e+03  3321.609733  1868.757186   
2350  127834.032492   7974.941494  8.434648e+06  2149.977677  3551.468192   
564   195909.364040  14484.050531  1.841882e+04  2192.591438  1750.690303   

                                                                           
                10           11            12            13            14  
849   1.933377e+04  2344.701448   1760.464133   7199.259852   6763.489654  
1149  3.727813e+04  7503.694361  12645.343429    595.948926    742.993571  
368   9.290002e+03  4936.422846  42312.987941   6968.938385  39961.441923  
2350  1.561951e+06  1532.875564    598.247116   5996.391338  24514.775674  
564   4.742312e+03  4621.943464  23969.949341  12679.214341   9308.293527

In [514]:
#step 2b: felony arrests
order = arima_dict['total_felony_arrest']['order']
seasonal_order = arima_dict['total_felony_arrest']['seasonal_order']
y = pd_data['ln_arrest'].dropna()
exog = pd_data['ln_crime'].dropna()
exog_pred = np.log(arima_dict['total_felony_crimes']['prediction'])
model = SARIMAX(y, order=order, seasonal_order=seasonal_order, exog = exog)
model_fit = model.fit(disp=False)
#determine predictions lengths and save the output in dictionary
n_steps = 12 + missing_n_arrest
# Generate in-sample predictions
y_pred = model_fit.fittedvalues
# Generate out-of-sample forecast_exog
forecast = model_fit.get_forecast(steps=n_steps, exog = exog_pred)
y_forecast_log = forecast.predicted_mean
y_forecast = np.exp(y_forecast_log)
#store forecast in the auto arima dictionary
#however, we want to make sure the length of the crimes and arrest dataset are equal. If so we can just
#save the y_forecast variable to the dictionary as is. However, if they differ in length we will want to 
#adjust the length of the signal and the prediction
if len(pd_data) == len(adm_df):
    arima_dict['total_felony_arrest']['y'] = np.exp(y)
    arima_dict['total_felony_arrest']['prediction'] = y_forecast
elif len(pd_data) < len(adm_df):
    diff_n = len(adm_df) - len(pd_data)
    arima_dict['total_felony_arrest']['y'] = pd.concat([np.exp(y),y_forecast[0:diff_n]],ignore_index=True)
    arima_dict['total_felony_arrest']['prediction'] = y_forecast[diff_n:]
elif len(pd_data) > len(adm_df):
    diff_n = len(pd_data) - len(adm_df)
    #take away from the y in order to have same dimensions as the arrest data in the next step
    arima_dict['total_felony_arrest']['y'] = np.exp(y)[0:-diff_n]
    #add actual values to the prediction
    arima_dict['total_felony_arrest']['prediction'] = pd.concat([np.exp(y)[-diff_n:],y_forecast],ignore_index=True)

In [515]:
arima_dict['total_felony_arrest']['y']

0     7630.000000
1     7866.000000
2     7918.000000
3     7505.000000
4     8091.000000
         ...     
92    7746.000000
93    9013.000000
94    8837.000000
95    8578.848151
96    8731.933956
Length: 97, dtype: float64

In [516]:
arima_dict['total_felony_arrest']['prediction']

97     8954.340619
98     8867.738048
99     8798.050179
100    8880.054392
101    8829.968709
102    8921.843282
103    8658.901091
104    8665.176196
105    8925.359032
106    8898.520572
107    8885.596880
108    8938.093181
109    9011.288965
Name: predicted_mean, dtype: float64

In [517]:
print(model_fit.summary())

                                     SARIMAX Results                                      
Dep. Variable:                          ln_arrest   No. Observations:                   95
Model:             SARIMAX(2, 1, 0)x(1, 0, 0, 12)   Log Likelihood                  94.228
Date:                            Fri, 21 Jun 2024   AIC                           -178.457
Time:                                    15:36:16   BIC                           -165.740
Sample:                                         0   HQIC                          -173.320
                                             - 95                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ln_crime       0.6025      0.143      4.222      0.000       0.323       0.882
ar.L1         -0.2548      0.148   

In [518]:
y_forecast

95     8578.848151
96     8731.933956
97     8954.340619
98     8867.738048
99     8798.050179
100    8880.054392
101    8829.968709
102    8921.843282
103    8658.901091
104    8665.176196
105    8925.359032
106    8898.520572
107    8885.596880
108    8938.093181
109    9011.288965
Name: predicted_mean, dtype: float64

In [519]:
#now we will iterate throught the crime simulation output and simulate the arrest
#predictions using those simulations as the exogenous variables
num_simulations = len(crime_simulation)
#define df to hold 1000 arrest forecasts
arrest_simulation = pd.DataFrame(index = range(num_simulations), columns = range(n_steps))

for i in range(0,len(crime_simulation)):
    exog_data = np.log(crime_simulation.iloc[i]).values.reshape(-1, 1)  # Ensure the shape is correct
    sim = model_fit.simulate(nsimulations=n_steps, repetitions=1, anchor='end', exog=exog_data)
    arrest_simulation.iloc[i] = np.exp(sim.values.flatten())

In [520]:
arrest_simulation.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
64,6053.647448,4217.493526,5869.048539,6703.092574,6741.336426,6116.31389,12089.288143,9508.325222,9697.082289,5790.517475,6963.330888,7563.200648,7345.920039,7648.692323,5507.706104
1702,468.676217,540.670974,16261.933082,4381.475133,3937.429844,133901.637747,5991.621063,79639.428844,7797.941686,1556.462408,44863.67643,8588.788816,5486.930438,9059.829965,13847.833949
1714,418.800747,416.818267,14261.314976,4472.129098,3700.718,131922.392585,6220.035288,118428.99524,8088.084662,2167.847605,52419.509304,8465.935101,9032.893701,14659.706837,24594.89854
196,1784.280852,1950.551818,4144.773975,12683.43871,12536.911119,10414.82788,12068.295548,5030.876865,7252.486671,3177.010803,6452.761576,7736.090326,6695.738135,9431.504186,7160.999295
938,2665.44476,619.726037,20404.393629,13309.922094,14566.1812,22069.003717,20386.676413,8432.452176,1749.171491,1224.581469,11542.443187,3422.224618,3987.483235,1922.287951,1790.445102


In [521]:
#step 2c: admission count
order = arima_dict['admission_count']['order']
seasonal_order = arima_dict['admission_count']['seasonal_order']
y = adm_df['ln_adm'].dropna()
exog = np.log(arima_dict['total_felony_arrest']['y'])
exog_pred = np.log(arima_dict['total_felony_arrest']['prediction'])
model = SARIMAX(y, order=order, seasonal_order=seasonal_order, exog = exog)
model_fit = model.fit(disp=False)
#determine predictions lengths and save the output in dictionary
n_steps = 12 + missing_n_adm
# Generate in-sample predictions
y_pred = model_fit.fittedvalues
# Generate out-of-sample forecast_exog
forecast = model_fit.get_forecast(steps=n_steps, exog = exog_pred)
y_forecast_log = forecast.predicted_mean
y_forecast = np.exp(y_forecast_log)
#store forecast in the auto arima dictionary
#however, we want to make sure the length of the crimes and arrest monthly_popset are equal. If so we can just
#save the y_forecast variable to the dictionary as is. However, if they differ in length we will want to 
#adjust the length of the signal and the prediction
if len(adm_df) == len(monthly_pop):
    arima_dict['admission_count']['y'] = np.exp(y)
    arima_dict['admission_count']['prediction'] = y_forecast
elif len(adm_df) < len(monthly_pop):
    diff_n = len(monthly_pop) - len(adm_df)
    arima_dict['admission_count']['y'] = pd.concat([np.exp(y),y_forecast[0:diff_n]],ignore_index=True)
    arima_dict['admission_count']['prediction'] = y_forecast[diff_n:]
elif len(adm_df) > len(monthly_pop):
    diff_n = len(adm_df) - len(monthly_pop)
    #take away from the y in order to have same dimensions as the arrest monthly_pop in the next step
    arima_dict['admission_count']['y'] = np.exp(y)[0:-diff_n]
    #add actual values to the prediction
    arima_dict['admission_count']['prediction'] = pd.concat([np.exp(y)[-diff_n:],y_forecast],ignore_index=True)


In [522]:
print(model_fit.summary())

                                     SARIMAX Results                                      
Dep. Variable:                             ln_adm   No. Observations:                   97
Model:             SARIMAX(0, 1, 0)x(1, 0, 0, 12)   Log Likelihood                 109.127
Date:                            Fri, 21 Jun 2024   AIC                           -212.253
Time:                                    15:36:35   BIC                           -204.560
Sample:                                         0   HQIC                          -209.144
                                             - 97                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
0              0.8136      0.037     22.063      0.000       0.741       0.886
ar.S.L12       0.0911      0.105   

In [523]:
#now we will iterate throught the crime simulation output and simulate the arrest
#predictions using those simulations as the exogenous variables
num_simulations = len(arrest_simulation)
#define df to hold 1000 arrest forecasts
adm_simulation = pd.DataFrame(index = range(num_simulations), columns = range(n_steps))
# Convert entire DataFrame to numeric
arrest_simulation = arrest_simulation.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
# y = adm_df['admission_count'].dropna()
diff_n = len(adm_df) - len(pd_data)

for i in range(0,len(arrest_simulation)):
    exog_data = np.log(arrest_simulation.iloc[i]).values.reshape(-1, 1)  # Ensure the shape is correct
    sim = model_fit.simulate(nsimulations=n_steps, repetitions=1, anchor='end', exog=exog_data[diff_n:])
    adm_simulation.iloc[i] = np.exp(sim.values.flatten())

In [524]:
arrest_simulation.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2.500000e+03,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,1410.453829,900.316711,10456.097167,6632.334437,7489.841344,59668.186740,9984.394238,1.904544e+05,4826.516238,2574.846608,38540.520747,4982.060493,7621.494115,7593.887100,8965.316560
std,1248.289378,1097.461947,6802.930557,6728.315582,5295.142556,48252.678512,7422.711500,2.937265e+05,2660.368495,1753.013156,43976.242932,2885.010957,6222.186240,4251.462220,6207.938547
min,211.556693,38.508849,1778.424552,228.004273,901.401220,5201.099348,1393.445947,2.528412e+03,400.453371,364.144125,2122.728931,623.964585,575.826044,551.919679,821.602044
25%,709.281740,255.310581,4866.717984,1113.888734,3550.438571,23314.804588,4687.149451,1.157228e+04,2830.516828,1306.356665,8079.056032,2646.540145,3193.258756,4466.722166,4209.524739
50%,1130.968756,645.696448,8495.969573,3470.677171,6106.564298,39806.393864,7383.922186,2.358928e+04,4375.403293,2204.168342,18448.869052,4445.316482,5859.099981,6998.909890,7866.655134
75%,1613.050910,1013.922420,14445.286899,10941.621514,10055.171165,95305.463713,12926.057538,2.931614e+05,6420.595146,3346.462670,51405.038096,6784.090930,9770.475045,10338.304001,11957.062079
max,9570.493594,8519.650007,35171.078657,31568.667347,33292.121534,226215.096056,47172.526948,1.501406e+06,17344.298601,13880.407985,254422.551887,16496.962530,42146.160326,25620.563901,40630.866760


In [525]:
adm_simulation.sample(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12
1668,3687.47176,820.034273,871.361966,17415.289929,1004.116514,12386.592237,1396.02174,458.663858,5194.150573,1477.246674,1216.932876,1726.266508,1932.659385
1625,5250.524159,671.142233,995.446508,10229.130566,829.801844,15050.576324,1020.632045,717.30923,9018.000443,1451.53825,1336.647455,1592.627018,1243.152947
2469,807.146877,188.988968,965.28657,2972.39744,1585.049079,120473.175739,330.753093,1869.217923,33787.427371,1545.773535,1684.768663,3569.910478,8723.063323
1082,2637.338604,919.740863,2956.245739,3195.062387,3422.92613,3027.146506,857.584881,172.941311,3227.44077,1757.00403,1811.326516,423.124716,621.51945
1561,3100.250882,696.43882,1057.623317,15183.588345,870.62817,12769.903486,1261.139417,824.221387,6280.058752,1580.098543,3691.796886,2622.328718,1029.378469


In [526]:
adm_simulation = adm_simulation.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
adm_simulation.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,2217.226431,1476.675020,1691.317359,9054.830115,2133.076530,20956.471350,1217.801504,722.872965,6221.504748,1256.430087,1745.018538,1787.444605,2031.807021
std,1197.984506,1298.341686,1000.070338,6368.986567,1310.869384,28898.647453,628.818647,428.606024,6099.566949,697.413347,1254.544548,990.900775,1320.949419
min,485.781470,74.134335,229.975899,861.120374,372.594804,657.388149,123.092682,110.406801,554.808780,210.279203,160.978338,212.153780,248.207114
25%,1220.411088,375.731595,939.248539,4237.547898,1182.741483,2421.977815,771.238330,406.142961,1853.209288,719.027932,848.428552,1047.960137,1042.018697
50%,1925.763928,955.156051,1461.187627,6660.165480,1733.457341,4556.752610,1109.805317,640.448163,3731.960033,1114.058687,1400.776224,1648.487819,1771.447530
75%,2943.316259,2351.515306,2231.870027,13604.877289,2739.487744,32892.522168,1574.665352,922.408135,8624.073749,1638.891860,2273.528387,2357.003954,2653.771489
max,6698.052516,6267.576404,7992.283757,33335.049271,9238.254169,157579.622735,4357.775506,3472.753129,45923.692947,4360.864160,9395.279394,7747.698694,11031.316481


In [527]:
#finally let's simulate adp
#step 3: fitting and predicting ADP for 12 mos
order = arima_dict['ADP']['order']
seasonal_order = arima_dict['ADP']['seasonal_order']
y = np.log(monthly_pop['ADP'].dropna())
#go without the covid flag for now
exog = np.log(arima_dict['admission_count']['y'])
#prepare the exog predictions
exog_pred = np.log(arima_dict['admission_count']['prediction'])
#predict
model = SARIMAX(y,
                order=order,
                seasonal_order=seasonal_order,
                exog = exog
                )
model_fit = model.fit()
IS_pred = model_fit.predict()
IS_absolute_error = abs(y - IS_pred)
pred = model_fit.get_forecast(steps=12, exog = exog_pred)

In [528]:
#now we will iterate throught the crime simulation output and simulate the arrest
#predictions using those simulations as the exogenous variables
num_simulations = len(adm_simulation)
#define df to hold 1000 arrest forecasts
adp_simulation = pd.DataFrame(index = range(num_simulations), columns = range(12))


for i in range(0,len(adm_simulation)):
    exog_data = np.log(adm_simulation.iloc[i]).values.reshape(-1, 1)  # Ensure the shape is correct
    sim = model_fit.simulate(nsimulations=12, repetitions=1, anchor='end', exog=exog_data[1:])
    adp_simulation.iloc[i] = np.exp(sim.values.flatten())

In [529]:
adp_simulation = adp_simulation.applymap(lambda x: pd.to_numeric(x, errors='coerce'))
adp_simulation.describe()

,0,1,2,3,4,5,6,7,8,9,10,11
count,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000,2500.000000
mean,5878.794980,6174.546433,7415.051842,6392.100271,7670.705752,6089.711479,5788.386325,7247.367350,6298.675669,6581.359362,6758.071387,6970.639869
std,711.396581,515.003336,826.076196,789.534129,1656.698362,1106.188666,1264.447182,1977.736607,1883.536496,2285.940462,2635.538816,3088.352086
min,4210.680502,4546.344659,4921.336560,4190.531681,4260.739281,3468.116019,2619.746172,2967.554375,2203.517989,1834.344831,1746.012859,1599.477018
25%,5306.078939,5811.756098,6829.290903,5854.567335,6403.034028,5316.784708,4888.655476,5831.121271,4953.140793,4955.364924,4880.982821,4751.956762
50%,5884.556881,6159.093743,7363.871311,6328.321652,7433.422776,5996.211191,5669.331349,7029.892845,5999.549642,6207.392673,6272.991861,6388.719338
75%,6471.930311,6514.077384,7964.281655,6906.775876,8717.905627,6772.264791,6552.928254,8402.579477,7345.029509,7791.922513,8187.031213,8500.085725
max,7591.000482,8082.707044,10671.135887,9510.088945,14425.446565,10485.902176,12334.023553,20105.493249,16467.996684,20704.228490,23682.182796,29636.433051


In [530]:
percentiles = np.arange(0.1, 1.0, 0.1)
quantiles = adp_simulation.quantile(percentiles)

# Transpose the result for better readability similar to df.describe()
quantiles_transposed = quantiles.T

#repeat for quartiles
percentiles_4 = np.arange(0.25, 1.0, 0.5)
quartiles = adp_simulation.quantile(percentiles_4)

# Transpose the result for better readability similar to df.describe()
quartiles_transposed = quartiles.T

# Calculate mean and std
mean = adp_simulation.mean()
std = adp_simulation.std()

# Combine mean, std, and quantiles into one DataFrame
stats_df = pd.concat([mean, std, quantiles_transposed,quartiles_transposed], axis=1)
stats_df.columns = ['mean', 'std'] + [f'{int(p*100)}%' for p in percentiles] + [f'{int(q*100)}%' for q in percentiles_4]

# Display the result
stats_df

,mean,std,10%,20%,30%,40%,50%,60%,70%,80%,90%,25%,75%
0,5878.794980,711.396581,4853.271085,5143.000936,5491.543528,5708.886636,5884.556881,6126.935778,6374.035556,6571.011023,6800.225198,5306.078939,6471.930311
1,6174.546433,515.003336,5516.873993,5736.236360,5886.633565,6026.862028,6159.093743,6293.413470,6440.806545,6606.794218,6854.091575,5811.756098,6514.077384
2,7415.051842,826.076196,6375.452729,6678.207879,6948.674815,7160.038492,7363.871311,7586.296912,7831.920554,8110.543042,8503.837301,6829.290903,7964.281655
3,6392.100271,789.534129,5410.321086,5738.696011,5956.185099,6130.497002,6328.321652,6536.045430,6758.501417,7052.770242,7446.491361,5854.567335,6906.775876
4,7670.705752,1656.698362,5751.991690,6218.075503,6599.607304,6990.716383,7433.422776,7872.572328,8378.335614,9093.762628,10029.164747,6403.034028,8717.905627
5,6089.711479,1106.188666,4773.860752,5165.197620,5448.670948,5713.070222,5996.211191,6285.531662,6597.032063,7004.090645,7553.529792,5316.784708,6772.264791
6,5788.386325,1264.447182,4304.427514,4692.034210,5081.215684,5356.708676,5669.331349,5978.263150,6353.374195,6753.792213,7468.102835,4888.655476,6552.928254
7,7247.367350,1977.736607,4972.226509,5582.964065,6048.898960,6522.413479,7029.892845,7505.187441,8020.353957,8770.693449,9905.282777,5831.121271,8402.579477
8,6298.675669,1883.536496,4164.372367,4745.060613,5175.391247,5583.392287,5999.549642,6484.808502,7061.331346,7704.576081,8912.897768,4953.140793,7345.029509
9,6581.359362,2285.940462,4037.864248,4700.898839,5182.090108,5698.156924,6207.392673,6799.395783,7420.529720,8227.569045,9678.945382,4955.364924,7791.922513


**n_simulations = 1,000 for the four variables, therefore 4,000 simulations were ran.**

Run 1 = avg final data point is 7197.748807

Run 2 = avg final data point is 6046.645771 

Run 3 = avg final data point is 7228.095617

**n_simulations = 2,500 for the four variables, therefore 10,000 simulations were ran.**

Run 1 = avg final data point is 6328.543542

Run 2 = avg final data point is 7408.970962

Run 3 = avg final data point is 6970.639869

In [531]:
#lets just look at the simulation output when we keep the admission exog input constant
num_simulations = 10000
#define df to hold 1000 arrest forecasts
adp_simulation_static_exog = pd.DataFrame(index = range(num_simulations), columns = range(12))
#define exog 
exog_data = np.log(arima_dict['admission_count']['prediction'])  # Ensure the shape is correct
sim = model_fit.simulate(nsimulations=12, repetitions=12, anchor='end', exog=exog_data)
adp_simulation_static_exog = np.exp(sim)

In [532]:
adp_simulation_static_exog.T.describe()

,98,99,100,101,102,103,104,105,106,107,108,109
count,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000,12.000000
mean,6452.543363,6498.532963,6624.947353,6777.900984,6855.660572,7037.106849,7208.664555,7354.315047,7478.511485,7689.866584,7845.186320,8009.703815
std,159.653822,240.702664,345.222547,541.886730,781.304595,1067.108084,1451.641977,1832.427592,2217.466984,2785.213120,3350.046362,3848.870375
min,6163.775229,6051.102442,6228.188641,6190.169225,5990.105984,5980.651624,5943.858298,5743.245203,5665.230290,5607.627791,5525.205812,5528.117744
25%,6385.656894,6331.600661,6361.301162,6412.557216,6293.980454,6476.361000,6213.587323,6121.575380,6186.206115,6152.925077,6133.837601,5846.223594
50%,6436.912021,6504.987897,6593.790873,6689.125818,6707.384681,6640.469457,6827.205704,6883.209690,6771.546292,6686.562375,6818.441457,6710.460156
75%,6494.870902,6657.403267,6765.701391,6979.103921,7212.302436,7432.787936,7713.040377,7870.755043,8167.189718,8323.358512,8181.934798,8508.108212
max,6773.846258,6913.867455,7309.863782,8154.914068,8791.961297,9868.764091,11241.614948,12410.827849,13717.570410,15799.724796,17823.060490,19414.993529


This output is a bit too high as well
